In [ ]:
import sklearn.metrics._scorer
import sklearn.tree
import pandas as pd
from matplotlib import pyplot as plt

from xgboost import XGBClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

from sklearn.preprocessing import LabelEncoder

In [153]:
data = pd.read_csv("chronic_desease.csv")
data = data.drop("Name", axis=1)

In [154]:
le = LabelEncoder()
cols = ['Age', 'Gender', 'Smoker', 'RiskLevel']

for col in cols:
    data[col] = le.fit_transform(data[col])

In [155]:
X = data.drop("ChronicDisease", axis=1)
y = data["ChronicDisease"]


In [156]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# For Decision Tree

In [157]:
from sklearn.model_selection import ParameterGrid

parameters = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'splitter': ['best', 'random'],
    'max_depth': range(1, 11),
    'max_features': [None, 'sqrt', 'log2']
}

df_data = []
for params in ParameterGrid(parameters):
    clf = DecisionTreeClassifier(**params, random_state=42).fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    df_data.append({
        **params,
        "train_score": clf.score(X_train, y_train),
        "test_score": clf.score(X_test, y_test),
        "AccuracyScore": accuracy_score(y_test, y_pred)*100
    })

df_data = pd.DataFrame(df_data)

treemodel = DecisionTreeClassifier()
cv = GridSearchCV(treemodel, param_grid=parameters, cv=5, scoring='accuracy')
cv.fit(X_train, y_train)

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",DecisionTreeClassifier()
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'criterion': ['gini', 'entropy', ...], 'max_depth': range(1, 11), 'max_features': [None, 'sqrt', ...], 'splitter': ['best', 'random']}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'accuracy'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",5
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold

In [158]:
print("Best Parameter:",cv.best_params_)

Best Parameter: {'criterion': 'log_loss', 'max_depth': 7, 'max_features': None, 'splitter': 'random'}


In [159]:
df_data

,criterion,max_depth,max_features,splitter,train_score,test_score,AccuracyScore
0,gini,1,None,best,0.877333,0.872,87.2
1,gini,1,None,random,0.820000,0.848,84.8
2,gini,1,sqrt,best,0.820000,0.848,84.8
3,gini,1,sqrt,random,0.820000,0.848,84.8
4,gini,1,log2,best,0.820000,0.848,84.8
...,...,...,...,...,...,...,...
175,log_loss,10,None,random,1.000000,0.996,99.6
176,log_loss,10,sqrt,best,1.000000,0.968,96.8
177,log_loss,10,sqrt,random,0.996000,0.996,99.6
178,log_loss,10,log2,best,1.000000,0.968,96.8


In [160]:

from sklearn.tree import DecisionTreeClassifier
dc = DecisionTreeClassifier(criterion='gini', max_features=None, max_depth=6, splitter='random')
treemodel = dc.fit(X_train, y_train)
y_pred = treemodel.predict(X_test)


In [ ]:
accuracy_score(y_test, y_pred)*100

97.6

# For Random Forest

In [162]:
parameters = {
    'n_estimators': [50, 100],
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_depth': range(1, 11),
    'max_features': [None, 'sqrt', 'log2'],
}

df_data = []
for params in ParameterGrid(parameters):
    clf = RandomForestClassifier(**params, random_state=42).fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    df_data.append({
        **params,
        "train_score": clf.score(X_train, y_train),
        "test_score": clf.score(X_test, y_test),
        "AccuracyScore": accuracy_score(y_test, y_pred)*100
    })

df_data = pd.DataFrame(df_data)

model1 = RandomForestClassifier()
cv1 = GridSearchCV(model1, param_grid=parameters, cv=5, scoring='accuracy')
cv1.fit(X_train, y_train)

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",RandomForestClassifier()
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'criterion': ['gini', 'entropy', ...], 'max_depth': range(1, 11), 'max_features': [None, 'sqrt', ...], 'n_estimators': [50, 100]}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'accuracy'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",5
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and 

In [163]:
print("Best Parameter:",cv.best_params_)

Best Parameter: {'criterion': 'log_loss', 'max_depth': 7, 'max_features': None, 'splitter': 'random'}


In [164]:
df_data

,criterion,max_depth,max_features,n_estimators,train_score,test_score,AccuracyScore
0,gini,1,None,50,0.888000,0.888,88.8
1,gini,1,None,100,0.885333,0.892,89.2
2,gini,1,sqrt,50,0.874667,0.892,89.2
3,gini,1,sqrt,100,0.870667,0.880,88.0
4,gini,1,log2,50,0.874667,0.892,89.2
...,...,...,...,...,...,...,...
175,log_loss,10,None,100,1.000000,0.996,99.6
176,log_loss,10,sqrt,50,1.000000,0.996,99.6
177,log_loss,10,sqrt,100,1.000000,0.996,99.6
178,log_loss,10,log2,50,1.000000,0.996,99.6


In [165]:

rf = RandomForestClassifier(criterion='entropy', max_features='sqrt', max_depth=6, n_estimators=50)
model1 = rf.fit(X_train, y_train)
y_pred = model1.predict(X_test)


In [166]:
accuracy_score(y_test, y_pred)*100

99.6

# For Logistic Regresion

In [167]:
parameters = {
    'penalty': ['l1', 'l2'],
    'C': [0.01, 0.1, 1.0, 10.0, 100.0],
    'solver': ['liblinear', 'saga'],
    'max_iter': [100, 1000]
}

df_data = []
for params in ParameterGrid(parameters):
    clf = LogisticRegression(**params, random_state=42).fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    df_data.append({
        **params,
        "train_score": clf.score(X_train, y_train),
        "test_score": clf.score(X_test, y_test),
        "AccuracyScore": accuracy_score(y_test, y_pred)*100
    })

df_data = pd.DataFrame(df_data)

model2 = LogisticRegression()
cv2 = GridSearchCV(model2, param_grid=parameters, cv=5, scoring='accuracy')
cv2.fit(X_train, y_train)

C:\Users\Admin\AppData\Roaming\Python\Python314\site-packages\sklearn\linear_model\_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(
C:\Users\Admin\AppData\Roaming\Python\Python314\site-packages\sklearn\linear_model\_logistic.py:1160: UserWarning: Inconsistent values: penalty=l1 with l1_ratio=0.0. penalty is deprecated. Please use l1_ratio only.
  warnings.warn(
C:\Users\Admin\AppData\Roaming\Python\Python314\site-packages\sklearn\linear_model\_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penal

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",LogisticRegression()
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'C': [0.01, 0.1, ...], 'max_iter': [100, 1000], 'penalty': ['l1', 'l2'], 'solver': ['liblinear', 'saga']}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'accuracy'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",5
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and parameter candidate is displ

In [168]:
print("Best Parameter:",cv.best_params_)

Best Parameter: {'criterion': 'log_loss', 'max_depth': 7, 'max_features': None, 'splitter': 'random'}


In [169]:
df_data

,C,max_iter,penalty,solver,train_score,test_score,AccuracyScore
0,0.01,100,l1,liblinear,0.878667,0.892,89.2
1,0.01,100,l1,saga,0.721333,0.724,72.4
2,0.01,100,l2,liblinear,0.894667,0.904,90.4
3,0.01,100,l2,saga,0.721333,0.724,72.4
4,0.01,1000,l1,liblinear,0.878667,0.892,89.2
5,0.01,1000,l1,saga,0.721333,0.724,72.4
6,0.01,1000,l2,liblinear,0.894667,0.904,90.4
7,0.01,1000,l2,saga,0.721333,0.724,72.4
8,0.10,100,l1,liblinear,0.992000,0.996,99.6
9,0.10,100,l1,saga,0.721333,0.724,72.4


In [170]:

lr = LogisticRegression(C = 1.0, max_iter=100, penalty='l1', solver='liblinear')
model2 = lr.fit(X_train, y_train)
y_pred = model2.predict(X_test)

C:\Users\Admin\AppData\Roaming\Python\Python314\site-packages\sklearn\linear_model\_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(
C:\Users\Admin\AppData\Roaming\Python\Python314\site-packages\sklearn\linear_model\_logistic.py:1160: UserWarning: Inconsistent values: penalty=l1 with l1_ratio=0.0. penalty is deprecated. Please use l1_ratio only.
  warnings.warn(


In [178]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)*100

97.99823844983585

In [172]:
parameters = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'subsample': [0.7, 0.12]
}

df_data = []
for params in ParameterGrid(parameters):
    clf = XGBClassifier(**params, random_state=42).fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    df_data.append({
        **params,
        "train_score": clf.score(X_train, y_train),
        "test_score": clf.score(X_test, y_test),
        "AccuracyScore": accuracy_score(y_test, y_pred)*100
    })

df_data = pd.DataFrame(df_data)

model3 = XGBClassifier()
cv3 = GridSearchCV(model, param_grid=parameters, cv=5, scoring='accuracy')
cv3.fit(X_train, y_train)

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.","XGBClassifier...ree=None, ...)"
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'learning_rate': [0.01, 0.1, ...], 'max_depth': [3, 5, ...], 'n_estimators': [50, 100, ...], 'subsample': [0.7, 0.12]}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'accuracy'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",5
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and par

In [173]:
print("Best Parameter:",cv.best_params_)

Best Parameter: {'criterion': 'log_loss', 'max_depth': 7, 'max_features': None, 'splitter': 'random'}


In [174]:
df_data

,learning_rate,max_depth,n_estimators,subsample,train_score,test_score,AccuracyScore
0,0.01,3,50,0.70,0.981333,0.996,99.6
1,0.01,3,50,0.12,0.802667,0.816,81.6
2,0.01,3,100,0.70,0.996000,0.996,99.6
3,0.01,3,100,0.12,0.969333,0.992,99.2
4,0.01,3,200,0.70,0.996000,0.996,99.6
...,...,...,...,...,...,...,...
91,0.30,9,50,0.12,0.993333,0.996,99.6
92,0.30,9,100,0.70,1.000000,0.996,99.6
93,0.30,9,100,0.12,0.996000,0.996,99.6
94,0.30,9,200,0.70,1.000000,0.996,99.6


In [175]:
xgb = XGBClassifier(learning_rate= 0.1, max_depth = 3, n_estimators = 100, subsample = 0.7)
model3 = xgb.fit(X_train, y_train)
y_pred = model3.predict(X_test)


In [176]:
accuracy_score(y_test, y_pred)*100

99.6